# Working with topologies

- Ground-up construction
- OpenMM
- Loading PDB files
- custom substructure loading

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    ℹ️ OpenFF <code>Topology</code> objects are little more than collections (lists) of OpenFF <code>Molecule</code> objects!
</div>

For starters, let's look at the `Topology` docstring:

In [ ]:
from openff.toolkit import Topology

?Topology

## Ground-up construction


Topologies can always be assembled by constructing individual molecules and adding them together; these methods are for making common operations easier.

To convert a single `Molecule` to a `Topology`, you can use either `Molecule.to_topology()` or `Topology.from_molecules`

In [ ]:
from openff.toolkit import Molecule

ligand = Molecule.from_smiles("Fc1ccc(/C=C/c2cc(NCCCCN3CCOCC3)c3cc(Cl)ccc3n2)cn1")
ligand.generate_conformers(n_conformers=1)

ligand

In [ ]:
from viz import visualize_topology

topology = ligand.to_topology()

# Equivalent
topology = Topology.from_molecules(molecules=[ligand])

visualize_topology(topology)

From here we can add as many other molecules as we wish. For example, create a water molecule and it to this topology 100 times.

In [ ]:
water = Molecule.from_mapped_smiles("[H:2][O:1][H:3]")

for index in range(100):
    topology.add_molecule(water)

topology.n_molecules

<div class="alert alert-info" style="max-width: 500px; margin-left: auto; margin-right: auto;">
    ℹ️ Positions are <i>optional</i> in <code>Molecule</code> (any by extension <code>Topology</code>) objects, so visualizing this topology in 3D doesn't make sense. Using it in a simulation would requiring assigning positions using a tool like Packmol or PDBFixer.
</div>

Keeping in mind that topologies are just collections of molecules, we can look up individual molecules by index in the `Topology.molecule()` function.

In [ ]:
topology.molecule(0), topology.molecule(1), topology.molecule(-1)

In [ ]:
topology.molecule(0)

## Serialization

OpenFF topologies, like molecules, can be serialized using common dict-like serialization formats such as JSON, YAML, XML, etc. These files are somewhat human-readable and do not tend to be efficiently compressed on disk.

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    ℹ️ These files are best when written and read by the OpenFF Toolkit; other tools aren't likely to be able to parse these files. There is also a change, prior to a stable 1.0 version of the toolkit, that it may not read files written by a different version of the toolkit.
</div>

Let's write this topology out to a JSON file - then open it up in your favorite text editor, or explore it using IPython's fancy JSON explorer.

In [ ]:
with open("topology.json", "w") as file:
    file.write(topology.to_json())

In [ ]:
import json

from IPython.display import JSON

JSON(json.loads(topology.to_json()), expanded=False)

They key value of serialization is the ability to dump something from memory onto disk and read it back in later. This might seem trivial for this topology, which we could easily generate, but maybe you're working with a more complex topology or you want to move files around HPC resources.

In [ ]:
with open("topology.json") as file:
    loaded_topology = Topology.from_json(file.read())

assert loaded_topology.n_molecules == 101

loaded_topology.molecule(0)